# Stacking

Несмотря на то, что в открытом доступе существует довольно много реализаций стекинга, некоторые из которых даже представлены в виде библиотечных функций, лучше сделать собственную. Стекинг - не классический алгоритм решения задачи, а скорее набор правил для помощи в решении задачи другим алгоритмам.

Создайте собственную функцию для блендинга/классического стекинга моделей.

Ниже:

- models - список объектов базовых алгоритмов

- meta_alg - мета-алгоритм

data_train, targets_train, data_test, targets_test - тренировочные и тестовые признаки и целевые переменные

- test_size - размер тестовых данных для блендинга в промежутке (0, 1)

- cv - количество разбиений для кросс-валидации

In [12]:
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [7]:
def stacking(models, meta_alg, data_train, targets_train, data_test, targets_test=None, test_size=None, cv=5):
  
    

SyntaxError: unexpected EOF while parsing (<ipython-input-7-eb4d6e078cdb>, line 3)

### 1.
В следующей ячейке в теле функции определен условный оператор if-else. После elif вместо pass пропишите код из лекции с некоторыми новыми вставками в таком порядке: деление data_train и targets_train на тренировочные и валидационные данные с помощью функции train_test_split, где test_size=test_size из определения функции; определение матрицы meta_mtrx; цикл, в котором заполняется meta_mtrx: сначала строка, где модель обучается с помощью метода fit на тренировочных данных из разбиения, затем строка, где матрица заполняется значениями предсказаний моделей на валидационных данных. На этом цикл заканчивается.

После цикла добавьте строку с методом fit мета-алгоритма: обучите его на значениях полученной матрицы meta_mtrx и целевой переменной для валидационных данных.

Определите матрицу meta_mtrx_test. Напишите цикл, где эта матрица заполняется предсказаниями базовых моделей на тестовых данных data_test.

После цикла сделайте предсказания мета-алгоритма для значений матрицы meta_mtrx_test.

Дополните код еще одним оператором if, который будет проверять, существуют ли данные targets_test для проверки качества работы модели на тестовых данных: если targets_test не является None, тогда выведите метрику roc_auc_score для предсказаний мета-алгоритма на тестовых данных.

In [47]:
from sklearn.metrics import roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

def stacking(models, meta_alg, data_train, targets_train, data_test, targets_test=None, random_state=None, test_size=None, cv=5):
    train = data_train
    train_true = targets_train
    test = data_test
    test_true = targets_test
    
    if test_size is None:
        skf = StratifiedKFold(n_splits=cv, random_state=17, shuffle = True)
        skf.get_n_splits(train, train_true)
    elif test_size > 0 and test_size < 1:
        x_train, valid, y_train, valid_true = train_test_split(train, 
                                                      train_true, 
                                                      test_size=test_size, 
                                                      random_state=17)
        skf = None
    else:
        raise ValueError("test_size must be between 0 and 1")
        skf = None
    
    if skf is None:
        for model in models:
            model.fit(x_train, y_train)        
    else:
        for train_index, test_index in skf.split(train, train_true):
            x_train, x_test = train[train_index], train[test_index]
            y_train, y_test = train_true[train_index], train_true[test_index]
            for model in models:
                model.fit(x_train, y_train)
        
    
    meta_mtrx = np.empty((valid.shape[0], len(models))) # (кол-во объектов, 4 алгоритма)
    
    for n, model in enumerate(models):
        meta_mtrx[:, n] = model.predict(valid)
        predicted = model.predict(test)
        print(f'{n} auc: {roc_auc_score(test_true, predicted)}')
        
    meta = meta_alg
    meta_model = meta.fit(meta_mtrx, valid_true)
        
    meta_mtrx_test = np.empty((test.shape[0], len(models))) 
    for n, model in enumerate(models):
        meta_mtrx_test[:, n] = model.predict(test)
    
    meta_predict = meta.predict(meta_mtrx_test)
    print(f'Stacking AUC: {roc_auc_score(test_true, meta_predict)}')
    

### 2.
Теперь напишите код стекинга вместо pass после оператора if.

Сразу определите матрицу meta_mtrx. Напишите цикл для заполнения матрицы: сначала напишите строку, где каждый столбец метапризнаков (для каждой модели) заполняется с помощью функции cross_val_predict(base_algotithm, data_train, targets_train, cv, method='predict'); следующая строка - обучите каждый базовый алгоритм на полном тренировочном датасете.

После цикла обучите мета-алгоритм на матрице метапризнаков meta_mtrx. Определите матрицу meta_mtrx_test.

Напишите второй цикл, где матрица meta_mtrx_test заполняется предсказаниями базовых моделей на тестовых данных data_test.

После цикла сделайте предсказания мета-алгоритма для значений матрицы meta_mtrx_test.

In [34]:
for n, model in enumerate(models):
    meta_mtrx_test[:, n] = model.predict(test)
    
meta_predict = meta.predict(meta_mtrx_test)
print(f'Stacking AUC: {roc_auc_score(test_true, meta_predict)}')

NameError: name 'test' is not defined

### 3.
Базовая функция стекинга готова. Теперь проверим, как она работает. Ниже представлен уже знакомый нам датасет Titanic, разделенный на тренировочный и тестовый датасеты; предопределенные базовые алгоритмы и мета-алгоритм. Ваша задача - составить список базовых алгоритмов и запустить функцию в трех различных вариантах (при этом в каждом из них все значения data_train, targets_train, data_test, targets_test должны быть определены):

1. Вызвать исключение "test_size must be between 0 and 1".

2. Установить test_size=0.3; вывести AUC и массив полученных предсказаний.

3. Оставить test_size=None; вывести AUC и массив полученных предсказаний.

In [51]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier


titanic = pd.read_csv('C:\\Users\\Tarar\\titanic.csv')
targets = titanic.Survived
data = titanic.drop(columns='Survived')

x_train, x_test, y_train, y_test = train_test_split(data, 
                                                    targets,
                                                    train_size=0.8,
                                                    random_state=17)

knn = KNeighborsClassifier(n_neighbors=3)
lr = LogisticRegression(random_state=17)
svc = SVC(random_state=17)

meta = XGBClassifier(n_estimators=40)
models = [knn, lr, svc]


In [52]:
stacking(models=models, meta_alg=meta, data_train=x_train, targets_train=y_train, data_test=x_test, targets_test=y_test, test_size=0.7, cv=5)


0 auc: 0.6154044972861205
1 auc: 0.7237658309640734
2 auc: 0.5812871543034376
[03:34:48] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Stacking AUC: 0.7100672008270871
